In [18]:
%pip install -qU langchain openai tiktoken pinecone-client[grpc] pypdf chromadb

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      creating build
      creating build\temp.win-amd64-cpython-310
      creating build\temp.win-amd64-cpython-310\Release
      creating build\temp.win-amd64-cpython-310\Release\python_bindings
      "C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -IC:\Users\jbury_000\AppData\Local\Temp\pip-build-env-d4kdwmzv\overlay\Lib\site-packages\pybind11\include -IC:\Users\jbury_000\AppData\Local\Temp\pip-build-env-d4kdwmzv\overlay\Lib\site-packages\numpy\core\include -I./hnswlib/ "-IC:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\include" "-IC:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x6

In [2]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

OPENAI_API_KEY = open_file('openaiapikey.txt')
PINECONE_API_KEY = open_file('pineconekey.txt')

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("GNSS-book.pdf")
pages = loader.load_and_split()
len(pages)

In [11]:
import tiktoken

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tokenizer = tiktoken.get_encoding('cl100k_base')

tiktoken_len(pages[0].page_content)

28

In [13]:
total = 0
for page in pages:  
    total += tiktoken_len(page.page_content)  
total

49890

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,
        chunk_overlap=20,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

#chunks = text_splitter.split_text(inputText)

In [14]:
pages[0].metadata

{'source': 'GNSS-book.pdf', 'page': 0}

In [17]:
import pinecone

index_name = 'gnss-book'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-southeast1-gcp'
)

# we create a new index
pinecone.create_index(
    name=index_name,
    metric='dotproduct',
    dimension=1536  # 1536 dim of text-embedding-ada-002
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Sat, 06 May 2023 15:12:51 GMT', 'x-envoy-upstream-service-time': '935', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.
